In [1]:
!pip install unidecode -q
!pip install --quiet -U pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.6 MB/s eta 0:00:00


In [8]:
import numpy as np
import pandas as pd
import itertools
from unidecode import unidecode
import pinecone

pinecone.init(api_key= 'XX', environment='XX')

# Set a name for your index
index_name = 'city-products'

# Make sure service with the same name does not exist
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
pinecone.create_index(name=index_name, dimension=100)


index = pinecone.Index(index_name=index_name)
print(index)

products = pd.read_csv('https://raw.githubusercontent.com/ashwinkadam/DigitalMarketing-Algorithms-Project/main/Product_recom/products.csv')
# Get all of the items
all_items = [str(title) for title in products['name']]
    
# Replace non-ASCII characters in item IDs with ASCII equivalents
all_items_ascii = [unidecode(title) for title in all_items]
    
# Transform items into factors
items_factors = np.load('city_factors.npy')
    
# Prepare item factors for upload
items_to_insert = list(zip(all_items_ascii, items_factors[1:].tolist()))


In [9]:
def chunks(iterable, batch_size=100):
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

print('Index statistics before upsert:', index.describe_index_stats())

for e, batch in enumerate(chunks([(ii[:64],x) for ii,x in items_to_insert])):index.upsert(vectors=batch)

print('Index statistics after upsert:', index.describe_index_stats())

Index statistics before upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}
Index statistics after upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 27221}},
 'total_vector_count': 27221}
